In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_data = pd.read_csv('../data/archive/cross-val/mouth/mouthLabels.csv')

In [4]:
Y = train_data[['label']]

In [5]:
kf = KFold(n_splits = 5)

In [6]:
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

In [7]:
idg  = ImageDataGenerator(rescale=1./255)

In [8]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [9]:
def create_new_model():
    base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(224,224,3))
    model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [10]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = '/saved_models/v2-cross-val/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
	training_data = train_data.iloc[train_index]
	validation_data = train_data.iloc[val_index]
	
	train_data_generator = idg.flow_from_dataframe(training_data, directory = '../data/archive/', target_size=(224, 224),
						       x_col = "filename", y_col = "label",
						       class_mode = "binary", shuffle = True)
	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = '../data/archive/', target_size=(224, 224),
							x_col = "filename", y_col = "label",
							class_mode = "binary", shuffle = True)
	
	# CREATE NEW MODEL
	model = create_new_model()
	# COMPILE NEW MODEL
	model.compile(loss='binary_crossentropy',
		      optimizer='Adam',
		      metrics=['accuracy'])
	
	# CREATE CALLBACKS
	checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
	callbacks_list = [checkpoint]
	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
	history = model.fit(train_data_generator,
			    epochs=8,
			    callbacks=callbacks_list,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
	model.load_weights("/saved_models/v2-cross-val/model_"+str(fold_var)+".h5")
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])
	
	tf.keras.backend.clear_session()
	
	fold_var += 1

Found 1158 validated image filenames belonging to 2 classes.
Found 290 validated image filenames belonging to 2 classes.
Epoch 1/8
37/37 [==============================] - 84s 1s/step - loss: 0.3564 - accuracy: 0.8471 - val_loss: 8.5495 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /saved_models/v2-cross-val\model_1.h5
Epoch 2/8
37/37 [==============================] - 37s 991ms/step - loss: 0.0333 - accuracy: 0.9920 - val_loss: 0.3925 - val_accuracy: 0.8034

Epoch 00002: val_accuracy improved from 0.50000 to 0.80345, saving model to /saved_models/v2-cross-val\model_1.h5
Epoch 3/8
37/37 [==============================] - 44s 1s/step - loss: 0.0085 - accuracy: 0.9997 - val_loss: 0.0641 - val_accuracy: 0.9828

Epoch 00003: val_accuracy improved from 0.80345 to 0.98276, saving model to /saved_models/v2-cross-val\model_1.h5
Epoch 4/8
37/37 [==============================] - 54s 1s/step - loss: 0.0578 - accuracy: 0.9815 - val_loss: 0.0713 -

In [14]:
np.mean(VALIDATION_ACCURACY)

0.9924042463302613

In [16]:
test_generator = idg.flow_from_directory(
        '../data/test/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 47 images belonging to 2 classes.


In [17]:
model.evaluate(test_generator)

3/3 [==============================] - 4s 996ms/step - loss: 0.2244 - accuracy: 0.9149


[0.22435472905635834, 0.914893627166748]